In [66]:
import os
import pandas as pd

In [67]:
directory = '/Users/cornflex/Desktop/Project/T5/T5-CapstoneProject/WebScraping/Filter_Roads/start/Dataset/'

files = os.listdir(directory)

csv_files = [f for f in files if f.endswith('.csv')]

dfs = []
for csv_file in csv_files:
    file_path = os.path.join(directory, csv_file)
    df = pd.read_csv(file_path)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

In [68]:
gd = df.groupby(['road_name', 'Start_point', 'End_point']).agg({
    'delay_min': 'mean',
    'speed_kmh': 'mean',
    'duration_in_traffic_min': 'mean',
    'distance_km': 'mean',
    'traffic_condition': lambda x: x.mode()[0]  # Take the most frequent traffic condition
}).reset_index()

In [69]:
df = gd[(gd['distance_km'] <= 81) & (gd['distance_km'] >= 6)]

In [70]:
df.reset_index(drop=True, inplace=True)

In [71]:
df[['Start_point','End_point']].duplicated().sum()

249

In [72]:
df = df.drop_duplicates(subset=['Start_point','End_point'])

In [73]:
import ast

df['Start_point'] = df['Start_point'].apply(ast.literal_eval)
df['End_point'] = df['End_point'].apply(ast.literal_eval)


top_left = (24.985496, 46.467449)
bottom_right = (24.609640, 46.928374)

def clean_in_riyadh(row):
    road_start = row['Start_point']
    road_end = row['End_point']
    
    if (((road_start[0] <= top_left[0]) and (road_start[1] >= top_left[1])) and ((road_start[0] >= bottom_right[0]) and (road_start[1] <= bottom_right[1]))):
        return 'in'   
    
    elif (((road_end[0] <= top_left[0]) and (road_end[1] >= top_left[1])) and((road_end[0] >= bottom_right[0]) and (road_end[1] <= bottom_right[1]))):
        return 'in'   
    
    else:
        return None

df['outside_riyadh'] = df.apply(clean_in_riyadh,axis=1);

df.isna().sum()
df.dropna(inplace=True)

df.drop(columns=['outside_riyadh'],inplace=True)

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 309 entries, 2 to 662
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   road_name                309 non-null    object 
 1   Start_point              309 non-null    object 
 2   End_point                309 non-null    object 
 3   delay_min                309 non-null    float64
 4   speed_kmh                309 non-null    float64
 5   duration_in_traffic_min  309 non-null    float64
 6   distance_km              309 non-null    float64
 7   traffic_condition        309 non-null    object 
dtypes: float64(4), object(4)
memory usage: 21.7+ KB


In [84]:
# pd.reset_option('display.max_rows', None)

df.reset_index(drop=True, inplace=True)

In [85]:
# df.to_csv(directory+'cleaned_dataset.csv', index=False)